# Adding weights to the expressions

First, we verify our functions D and E are infact homometric

In [1]:
x, a, b, c, d = var('x a b c d')
#pretty sure this is only right for x a real - think a conjugate is needed otherwise
def adjoint(polynomial, n = 8):
    out(x) = 0 
    for elem in polynomial.coefficients(x):
        out += elem[0](x) * x^( mod(-1 * elem[1](x), n) )
    return out

def reduce_exp(polynomial, n = 8):
    out(x) = 0
    for elem in polynomial.coefficients(x):
        out += elem[0](x) * x^( mod(elem[1](x), n))
    return out

def patterson(polynomial, n = 8):
    adjoint_poly = adjoint(polynomial, n)
    return reduce_exp(polynomial * adjoint_poly, n)

In [2]:
D_orig(x) = 1 + x + x^3 + x^4
D_orig_star(x) = adjoint(D_orig, 8)
E_orig(x) = 1 + x^3 + x^4 + x^5
E_orig_star(x) = adjoint(E_orig, 8)

print("Before reducing the exponents:")
print((D_orig * D_orig_star)(x).full_simplify())
print((E_orig * E_orig_star)(x).full_simplify())

D_orig_combined(x) = reduce_exp(D_orig * D_orig_star, 8)
E_orig_combined(x) = reduce_exp(E_orig * E_orig_star, 8)
print("After reducing the exponents:")
print(D_orig_combined(x))
print(E_orig_combined(x))

print("Computed all at once:")
print(patterson(D_orig, 8)(x))
print(patterson(E_orig, 8)(x))

Before reducing the exponents:
x^11 + x^10 + x^9 + 3*x^8 + 2*x^7 + x^6 + 2*x^5 + 2*x^4 + x^3 + x + 1
x^10 + 2*x^9 + 3*x^8 + 2*x^7 + x^6 + 2*x^5 + 2*x^4 + 2*x^3 + 1
After reducing the exponents:
2*x^7 + x^6 + 2*x^5 + 2*x^4 + 2*x^3 + x^2 + 2*x + 4
2*x^7 + x^6 + 2*x^5 + 2*x^4 + 2*x^3 + x^2 + 2*x + 4
Computed all at once:
2*x^7 + x^6 + 2*x^5 + 2*x^4 + 2*x^3 + x^2 + 2*x + 4
2*x^7 + x^6 + 2*x^5 + 2*x^4 + 2*x^3 + x^2 + 2*x + 4


### Basic investigation

We now consider what happens if we multiply the x^3 term in D with weight 3. Are there a, b, c, d weights to add to E that make the modified D and E homometric?

In [3]:
D(x) = 1 + x + 3*x^3 + x^4
D_star(x) = adjoint(D, 8)
D_combined(x) = patterson(D, 8)


E(x) = a + b*x^3 + c*x^4 + d*x^5
E_star(x) = adjoint(E, 8)
E_combined(x) = patterson(E, 8)


print(D_combined(x))
print(E_combined(x))

4*x^7 + 3*x^6 + 4*x^5 + 2*x^4 + 4*x^3 + 3*x^2 + 4*x + 12
b*d*x^6 + (b*c + c*d)*x^7 + 2*a*c*x^4 + (a*b + a*d)*x^5 + b*d*x^2 + (a*b + a*d)*x^3 + a^2 + b^2 + c^2 + d^2 + (b*c + c*d)*x


What we got manually reducing:

In [4]:
print((D * D_star)(x).full_simplify())
#manually replacing exponents
D_manual(x) = x^3 + 3*x^2 + 3*x^1 + 11*x^0 + 4*x^7 + 3*x^6 + 4*x^5 + 2*x^4 + 3*x^3 + x + 1
bool(D_manual(x) == D_combined(x))

x^11 + 3*x^10 + 3*x^9 + 11*x^8 + 4*x^7 + 3*x^6 + 4*x^5 + 2*x^4 + 3*x^3 + x + 1


True

and for E this gives

In [5]:
print((E * E_star)(x).full_simplify())
#manually replacing exponents
E_manual(x) =  b*d*x^2 + (b*c + c*d)*x^1 + (b^2 + c^2 + d^2)*x^0 + b*d*x^6 + (b*c + c*d)*x^7 + 2*a*c*x^4 + (a*b + a*d)*x^5 + (a*b + a*d)*x^3 + a^2
bool(E_combined(x) == E_manual(x))

b*d*x^10 + (b*c + c*d)*x^9 + (b^2 + c^2 + d^2)*x^8 + b*d*x^6 + (b*c + c*d)*x^7 + 2*a*c*x^4 + (a*b + a*d)*x^5 + (a*b + a*d)*x^3 + a^2


True

Note that if they are homometric, we get the following equations:

0. a^2 + b^2 + c^2 + d^2 = 12 
1. bc + cd = 4
2. bd = 3
3. (ab + ad) = 4
4. 2ac = 2
5. (ab + ad) = 4
6. bd = 3
7. bc + cd = 4 

Suppose a,b,c,d in R.

Combine equations 1 and 3 to get c(b+d) = 4 = a(b+d). b+d != 0 or else this equation isn't true, so we have c=a and ac = 1, so a = +1, -1, c = +1, -1. 

If c = +1, then b+d = 4, bd = 3, so 3 = (4-d)d = 4d - d^2 implies d^2 - 4d + 3 = (d-3)(d-1) = 0, so d = 3 or 1, which means b = 1 or 3.

If c = -1, then b+d = -4, bd = 3, so 3 = (-4-d)d = -4d - d^2 implies d^2 + 4d + 3 = (d+3)(d+1) = 0, so d = -3 or -1, which means b = -1 or -3. 

So the viable solutions seem to be 
- a = 1,  b = 1,  c = 1,  d = 3
- a = 1,  b = 3,  c = 1,  d = 1
- a = -1, b = -1, c = -1, d = -3
- a = -1, b = -3, c = -1, d = -1

We can also just get this directly through sage.

In [6]:
print(solve([a^2 + b^2 + c^2 + d^2 == 12, b*c+c*d == 4, b*d == 3, a*b+a*d == 4, 2*a*c ==2], a, b, c, d))

[
[a == -1, b == -3, c == -1, d == -1],
[a == 1, b == 1, c == 1, d == 3],
[a == -1, b == -1, c == -1, d == -3],
[a == 1, b == 3, c == 1, d == 1]
]


Here is a quick verification that these weights work.

In [7]:
E_guess1(x) = 1 + 1*x^3 + 1*x^4 + 3*x^5
print(patterson(E_guess1, 8)(x))

E_guess2(x) = 1 + 3*x^3 + 1*x^4 + 1*x^5
print(patterson(E_guess2, 8)(x))

E_guess3(x) = -1 - 1*x^3 - 1*x^4 - 3*x^5
print(patterson(E_guess3, 8)(x))

E_guess4(x) = -1 - 3*x^3 - 1*x^4 - 1*x^5
print(patterson(E_guess4, 8)(x))

print(D_combined(x))


4*x^7 + 3*x^6 + 4*x^5 + 2*x^4 + 4*x^3 + 3*x^2 + 4*x + 12
4*x^7 + 3*x^6 + 4*x^5 + 2*x^4 + 4*x^3 + 3*x^2 + 4*x + 12
4*x^7 + 3*x^6 + 4*x^5 + 2*x^4 + 4*x^3 + 3*x^2 + 4*x + 12
4*x^7 + 3*x^6 + 4*x^5 + 2*x^4 + 4*x^3 + 3*x^2 + 4*x + 12
4*x^7 + 3*x^6 + 4*x^5 + 2*x^4 + 4*x^3 + 3*x^2 + 4*x + 12


Question - If we have some weights A,B,C,D applied to D - is the ones term also be in the form A^2 + B^2 + C^2 + D^2? It turns out yes! So we have a condition that must hold. The sum of the squares of the weights must be equal for both.

If we have D = Ax^(k_1) + Bx^(k_2) + Cx^(k_3) + Dx^(k_4) where each k is distinct, its clear why this A^2 + B^2 + C^2 + D^2 term appears - in the definition of D^*, we invert the powers, so when we multiply them together these appear. Furthermore, since k's are distinct this is the only way to get x^0 terms (I think). So this probably works not only for arbitrary D, but also with more x's as well.

Guess - when we added weight 3 to x^3, it affected a vertex which had distances 1/2/3 in the Patterson diagram. The subsequent solutions (in positive integers) added weight 3 to either x^3 or x^5 - note both of these also have distances 1/2/3. Perhaps for positive integer weights, we can add "extra points" on top of each other?

In particular, it might mean there are no solutions if we add a weight to 1 in D - say we make it 3 + x + x^3 + x^4. Notably, this vertex has distances 1/3/4, but there is no such vertex in E's Patterson diagram.

(TODO - add pictures of said diagrams here)

In [8]:
D_test(x) = 3+x+1*x^3+x^4
D_test_combined(x) = patterson(D_test, 8)
print(D_test_combined(x))
print(E_combined(x))


4*x^7 + x^6 + 4*x^5 + 6*x^4 + 4*x^3 + x^2 + 4*x + 12
b*d*x^6 + (b*c + c*d)*x^7 + 2*a*c*x^4 + (a*b + a*d)*x^5 + b*d*x^2 + (a*b + a*d)*x^3 + a^2 + b^2 + c^2 + d^2 + (b*c + c*d)*x


This gives the following equations:

0. a^2 + b^2 + c^2 + d^2 = 12
1. bc + cd = 4
2. bd = 1
3. ab + ad = 4
4. 2ac = 6
5. ab + ad = 4
6. bd = 1
7. bc + cd = 4

This has no solutions in the integers since equations 2 forces b=d=1 or b=d=-1. However, if b=d=1, then c(b+d) = 2c = 4 so c=2. Similarly, a(b+d) = 2a = 4 so a = 2, but 2^2 + 2^2 + 1^2 + 1^2 = 10 not 12. Similar issues arise when b=d=-1.

In the reals, Again, b, d, a, c are all not 0 and a/c and b/d have same signs by equations 2 and 4. c(b+d) = 4 = a(b+d), so b+d can't be zero and thus a=c. Since ac=3, this means a = c = sqrt(3) or a = c = -sqrt(3). Note that we can substitute this into 1 to get sqrt(3) (b+d) = 4 and bd = 1. Solving this for b/d gives us the following (along with the case of c = -sqrt(3)):

In [9]:
print(solve([sqrt(3)*(b+d) == 4, b*d == 1], b, d))
print(solve([-sqrt(3)*(b+d) == 4, b*d == 1], b, d))

[
[b == 1/3*sqrt(3), d == sqrt(3)],
[b == sqrt(3), d == 1/3*sqrt(3)]
]
[
[b == -sqrt(3), d == -1/3*sqrt(3)],
[b == -1/3*sqrt(3), d == -sqrt(3)]
]


Note that the above solution doesn't satisify 0. This is because we have 3 "sqrt(3)" terms and 1 "1/3 sqrt(3)" term, which gives a^2 + b^2 + c^2 + d^2 = 3 + 3 + 3 + 1/3 != 12. Thus this case doesn't have any real solution either. So perhaps the idea behind the duplicate vertex checks out?

In [10]:
#Alternatively, just do the solve all at once with sage
print(solve([a^2 + b^2 + c^2 + d^2 == 12, b*c+c*d == 4, b*d == 1, a*b+a*d == 4, 2*a*c ==6], a, b, c, d))

[

]


We can also try this for x and x^4

In [11]:
D_test(x) = 1+3*x+x^3+x^4
D_test_combined(x) = patterson(D_test, 8)

print(D_test_combined(x))
print(E_combined(x))
print(solve([a^2 + b^2 + c^2 + d^2 == 12, b*c+c*d == 4, b*d == 3, a*b+a*d == 4, 2*a*c ==2], a, b, c, d))


4*x^7 + 3*x^6 + 4*x^5 + 2*x^4 + 4*x^3 + 3*x^2 + 4*x + 12
b*d*x^6 + (b*c + c*d)*x^7 + 2*a*c*x^4 + (a*b + a*d)*x^5 + b*d*x^2 + (a*b + a*d)*x^3 + a^2 + b^2 + c^2 + d^2 + (b*c + c*d)*x
[
[a == -1, b == -3, c == -1, d == -1],
[a == 1, b == 1, c == 1, d == 3],
[a == -1, b == -1, c == -1, d == -3],
[a == 1, b == 3, c == 1, d == 1]
]


In [12]:
D_test(x) = 1+x+x^3+3*x^4
D_test_combined(x) = patterson(D_test, 8)

print(D_test_combined(x))
print(E_combined(x))
print(solve([a^2 + b^2 + c^2 + d^2 == 12, b*c+c*d == 4, b*d ==1, a*b+a*d == 4, 2*a*c == 6], a, b, c, d))

4*x^7 + x^6 + 4*x^5 + 6*x^4 + 4*x^3 + x^2 + 4*x + 12
b*d*x^6 + (b*c + c*d)*x^7 + 2*a*c*x^4 + (a*b + a*d)*x^5 + b*d*x^2 + (a*b + a*d)*x^3 + a^2 + b^2 + c^2 + d^2 + (b*c + c*d)*x
[

]


### Trying this but with variable weight on 1 variable

First we try with varying on 1

In [13]:
var('a b c d A')
D_test(x) = A+x+x^3+x^4
D_test_combined(x) = patterson(D_test, 8)

print(D_test_combined(x))
print(E_combined(x))
print(solve([a^2 + b^2 + c^2 + d^2 == 3 + A^2, b*c+c*d == A + 1, b*d == 1, a*b+a*d == A+1, 2*a*c == 2*A], a, b, c, d))

(A + 1)*x^7 + (A + 1)*x^5 + x^6 + 2*A*x^4 + A*x^3 + x^3 + A^2 + A*x + x^2 + x + 3
b*d*x^6 + (b*c + c*d)*x^7 + 2*a*c*x^4 + (a*b + a*d)*x^5 + b*d*x^2 + (a*b + a*d)*x^3 + a^2 + b^2 + c^2 + d^2 + (b*c + c*d)*x
[

]


Sage reports no solutions - but is this accurate? Lets gather the equations first and decide

0. a^2 + b^2 + c^2 + d^2 == 3 + A^2
1. bc+cd == A + 1
2. bd == 1
3. ab+ad == A+1
4. 2ac == 2A


Equations 1 and 3 give c(b+d) = a(b+d). Note that b+d = 0 but bd = 1 is a contradiction in the reals. Thus (b+d) != 0, so a=c. 

Thus from 4 we get ac = A so c^2 = a^2 = A. So a = c = +/- sqrt(A). 

Equation 3 gives us bd=1 so d = 1/b. Then equation 1 gives us c(b + 1/b) == A + 1 == sqrt(A)(b + 1 / b) = A + 1 => (b + 1/b) = sqrt(A) + 1 / sqrt(A). Thus the solutions for b are sqrt(A) and 1/sqrt(A) and d are the recriprocals. Note if c = -sqrt(A) instead, then you just add the minus sign to both sides.

Now, a^2 + c^2 + b^2 + d^2 = sqrt(A)^2 + sqrt(A)^2 + sqrt(A)^2 + 1/sqrt(A)^2 = 3A + 1/A = 3 + A^2.

(allegedly) this is only solvable when A=1. Thus any other weight has no solution?


In [14]:
solve(3+ A^2 == 3*A + 1/A, A)

[A == 1]

Next we try variation on x

In [15]:
var('a b c d A')
D_test(x) = 1+A*x+x^3+x^4
D_test_combined(x) = patterson(D_test, 8)
print(D_test_combined(x))
print(E_combined(x))
print(solve([a^2 + b^2 + c^2 + d^2 == 3 + A^2, b*c+c*d == A + 1, b*d == A, a*b+a*d == A+1, 2*a*c == 2], a, b, c, d))

(A + 1)*x^7 + A*x^6 + (A + 1)*x^5 + A*x^3 + 2*x^4 + A*x^2 + x^3 + A^2 + A*x + x + 3
b*d*x^6 + (b*c + c*d)*x^7 + 2*a*c*x^4 + (a*b + a*d)*x^5 + b*d*x^2 + (a*b + a*d)*x^3 + a^2 + b^2 + c^2 + d^2 + (b*c + c*d)*x
[
[a == -1, b == -A, c == -1, d == -1],
[a == 1, b == 1, c == 1, d == A],
[a == -1, b == -1, c == -1, d == -A],
[a == 1, b == A, c == 1, d == 1]
]


Next we try variation on x^3

In [16]:
var('a b c d A')
D_test(x) = 1+x+A*x^3+x^4
D_test_combined(x) = patterson(D_test, 8)
print(D_test_combined(x))
print(E_combined(x))
print(solve([a^2 + b^2 + c^2 + d^2 == 3 + A^2, b*c+c*d == A + 1, b*d == A, a*b+a*d == A+1, 2*a*c == 2], a, b, c, d))

(A + 1)*x^7 + A*x^6 + (A + 1)*x^5 + A*x^3 + 2*x^4 + A*x^2 + x^3 + A^2 + A*x + x + 3
b*d*x^6 + (b*c + c*d)*x^7 + 2*a*c*x^4 + (a*b + a*d)*x^5 + b*d*x^2 + (a*b + a*d)*x^3 + a^2 + b^2 + c^2 + d^2 + (b*c + c*d)*x
[
[a == -1, b == -A, c == -1, d == -1],
[a == 1, b == 1, c == 1, d == A],
[a == -1, b == -1, c == -1, d == -A],
[a == 1, b == A, c == 1, d == 1]
]


Lastly, we try variation on x^4

In [17]:
var('a b c d A')
D_test(x) = 1+x+x^3+A*x^4
D_test_combined(x) = patterson(D_test, 8)
print(D_test_combined(x))
print(E_combined(x))
print(solve([a^2 + b^2 + c^2 + d^2 == 3 + A^2, b*c+c*d == A + 1, b*d == 1, a*b+a*d == A+1, 2*a*c == 2*A], a, b, c, d))

(A + 1)*x^7 + (A + 1)*x^5 + x^6 + 2*A*x^4 + A*x^3 + x^3 + A^2 + A*x + x^2 + x + 3
b*d*x^6 + (b*c + c*d)*x^7 + 2*a*c*x^4 + (a*b + a*d)*x^5 + b*d*x^2 + (a*b + a*d)*x^3 + a^2 + b^2 + c^2 + d^2 + (b*c + c*d)*x
[

]


Again, we should probably verify if there really are no solutions w.r.t to A. Our equations are

0. a^2 + b^2 + c^2 + d^2 = A^2 + 3
1. bc+cd = A+1
2. bd = 1
3. ab + ad = A+1
4. 2ac = 2A

These equations are the same as before, so no new solutions (aside from when A = 1)

We now try variation on both x and x^3 at the same time - note this does have solutions. At least for these functions, it seems that IF you can vary the coefficient on a certain monomial by itself and there are solutions, and you can vary the coefficient on a different monomial by itself, you can vary the coefficients together and get a new homometric function for E

In [18]:
var('a b c d A B')
D_test(x) = 1+A*x+B*x^3+x^4
D_test_combined(x) = patterson(D_test, 8)
print(D_test_combined(x))
print(E_combined(x))
print(solve([a^2 + b^2 + c^2 + d^2 == 2 + A^2 + B^2, b*c+c*d == A+B, b*d == A*B, a*b+a*d == A+B, 2*a*c == 2], a, b, c, d))

A*B*x^6 + (A + B)*x^7 + (A + B)*x^5 + A*B*x^2 + A*x^3 + B*x^3 + 2*x^4 + A^2 + B^2 + A*x + B*x + 2
b*d*x^6 + (b*c + c*d)*x^7 + 2*a*c*x^4 + (a*b + a*d)*x^5 + b*d*x^2 + (a*b + a*d)*x^3 + a^2 + b^2 + c^2 + d^2 + (b*c + c*d)*x
[
[a == 1, b == B, c == 1, d == A],
[a == 1, b == A, c == 1, d == B],
[a == -1, b == -A, c == -1, d == -B],
[a == -1, b == -B, c == -1, d == -A]
]


We can verify the solutions above are all of them - we have ac = 1 and a(b+d) = c(b+d) = A+B, so we have a = c = +/- 1. If a=c=1, Then we have b+d = A+B and bd = AB => b = AB / d, so AB / d + d = A + B. There are 2 obvious solutions for d with d = A or B. This is quadratic, so up to 2 solutions. Similar solutions happen if a=c=-1. Hence the above solutions seem good!

Maybe we can get lucky with putting A on 1 and B on x and it'll work out?

In [19]:
var('a b c d A B')
D_test(x) = A+B*x+x^3+x^4
D_test_combined(x) = patterson(D_test, 8)
print(D_test_combined(x))
print(E_combined(x))

(A*B + 1)*x^7 + B*x^6 + (A + B)*x^5 + 2*A*x^4 + A*x^3 + B*x^3 + A*B*x + B*x^2 + A^2 + B^2 + x + 2
b*d*x^6 + (b*c + c*d)*x^7 + 2*a*c*x^4 + (a*b + a*d)*x^5 + b*d*x^2 + (a*b + a*d)*x^3 + a^2 + b^2 + c^2 + d^2 + (b*c + c*d)*x


It turns out to be some horrible symbolic mess - 
a^2 = (A^2 + AB)/(AB+1). You can then solve for b/c/d from this, but its pretty terrible. 

To get here - We have d = B/b and c = A/a. Apply those to c(b+d) = AB + 1 and a(b+d) = A+B to get 2 equations with only a and b. This gives 

1. a(b+ [B / b]) = A+B ==> A a (b + [B / b]) = A^2 + AB.
2. (A/a)(b + [B / b]) = AB + 1 ==> A a (b + [B / b]) = a^2(AB + 1).
3. Hence, a^2 = (A^2 + AB) / (AB + 1)

Solving for b probably involves some quadratic stuff - alternatively you can calculate b^2 + d^2 through either c(b+d) = AB + 1 and bd = B or a(b+d) = A + B. At this point probably use sage to handle the algebraic manipulation

### fully arbitrary stuff

In [20]:
var("a b c d A B C D")
E(x) = a + b*x^3 + c*x^4 + d*x^5
E_star(x) = adjoint(E, 8)
E_combined(x) = patterson(E, 8)

D(x) = A+B*x+C*x^3+D*x^4
print(patterson(D,8)(x))
print("Compare D_combined above to E_combined below")
print(E_combined(x))

B*C*x^6 + (A*B + C*D)*x^7 + 2*A*D*x^4 + (A*C + B*D)*x^5 + A*C*x^3 + B*D*x^3 + B*C*x^2 + A*B*x + C*D*x + A^2 + B^2 + C^2 + D^2
Compare D_combined above to E_combined below
b*d*x^6 + (b*c + c*d)*x^7 + 2*a*c*x^4 + (a*b + a*d)*x^5 + b*d*x^2 + (a*b + a*d)*x^3 + a^2 + b^2 + c^2 + d^2 + (b*c + c*d)*x


These equations are probably specific to this choice of D and E

0. a^2 + b^2 + c^2 + d^2 = A^2 + B^2 + C^2 + D^2 
1. bc+cd = AB + CD
2. bd = BC
3. ab + ad = AC + BD
4. 2ac = 2AD
5. ab + ad = AC + BD
6. bd = BC
7. bc + cd = AB + CD

interestingly - there is some symmetry here.

From 2, we have d = BC / b.
From 4, we have c = AD / a.

From 1, we have c(b+d) = (AD / a)(b + BC / b) = AB + CD. Note this means a(AD)(b + BC / b) = a^2(AB + CD).

From 3, we have a(b+d) = a(b + BC / b) = AC + BD. Note this means a(AD)(b + BC / b) = AD(AC + BD). 

Hence we have a^2(AB + CD) = AD(AC + BD), so a^2 = AD(AC+BD)/(AB + CD).

At this point, we should probably backsub to get to b. This will involve some quadratic stuff unfortunately. 

### random stuff

Quick test for arbitrary r as in Theorem 3.9. Lets try r = 1/12 on our example. It seems to create 2 homometric functions in Z_12.

In [21]:
#0  1/12,  1/4+1/12,  1/2
#0  1/12   4/12       6/12
D_weird(x) = 1 + x + x^4 + x^6
#0  1/12+1/4  1/2  1/12 + 1/2
#0  4/12      6/12 7/12 
E_weird(x) = 1 + x^4 + x^6 + x^7
print(patterson(D_weird, 12)(x))
print(patterson(E_weird, 12)(x))

x^11 + x^10 + x^9 + x^8 + x^7 + 2*x^6 + x^5 + x^4 + x^3 + x^2 + x + 4
x^11 + x^10 + x^9 + x^8 + x^7 + 2*x^6 + x^5 + x^4 + x^3 + x^2 + x + 4
